In [ ]:
import pandas as pd

### Продолжаем работу с датасетом, посвященном пассажирам Титаника

Скачать датасет и посмотреть его описание можно по адресу: https://www.kaggle.com/c/titanic/data  
Скачать без регистрации: https://drive.google.com/open?id=1zy9JKp-bTfF0gRYkGl-g0X4o72NAhl1u

In [ ]:
df = pd.read_csv('train.csv')

In [ ]:
df.head(10)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,alone
0,0,3,male,22.00000,1,0,7.25,S,1
1,1,1,female,38.00000,1,0,71.28,C,1
2,1,3,female,26.00000,0,0,7.92,S,0
3,1,1,female,35.00000,1,0,53.10,S,1
4,0,3,male,35.00000,0,0,8.05,S,0
5,0,3,male,25.14062,0,0,8.46,Q,0
6,0,1,male,54.00000,0,0,51.86,S,0
7,0,3,male,2.00000,3,1,21.08,S,1
8,1,3,female,27.00000,0,2,11.13,S,1
9,1,2,female,14.00000,1,0,30.07,C,1


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
Survived    891 non-null int64
Pclass      891 non-null int64
Sex         891 non-null object
Age         891 non-null float64
SibSp       891 non-null int64
Parch       891 non-null int64
Fare        891 non-null float64
Embarked    891 non-null object
dtypes: float64(2), int64(4), object(2)
memory usage: 55.8+ KB


# Изменение датафрейма

In [ ]:
df.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [ ]:
# Удаляем столбцы, в которых нет ценной для нас информации
df.drop(columns = ['PassengerId', 'Name', 'Ticket', 'Cabin'], axis = 1, inplace = True)

In [ ]:
# Округляем стоимость билета до двух знаков после запятой
df['Fare'] = round(df['Fare'], 2)

Заполняем пустые значения в столбце Embarked с помощью метода fillna()

In [ ]:
df['Embarked'].fillna('no data')

0      S
1      C
2      S
3      S
4      S
5      Q
6      S
7      S
8      S
9      C
10     S
11     S
12     S
13     S
14     S
15     S
16     Q
17     S
18     S
19     C
20     S
21     S
22     Q
23     S
24     S
25     S
26     C
27     S
28     Q
29     S
      ..
861    S
862    S
863    S
864    S
865    S
866    C
867    S
868    S
869    S
870    S
871    S
872    S
873    S
874    C
875    C
876    S
877    S
878    S
879    C
880    S
881    S
882    S
883    S
884    S
885    Q
886    S
887    S
888    S
889    C
890    Q
Name: Embarked, Length: 891, dtype: object

In [ ]:
df['Embarked'].fillna(df['Embarked'].value_counts().index[0], inplace = True)

Другие варианты заполнения пустых значений можно посмотреть в параметрах метода fillna в документации: http://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.fillna.html

### Заполняем пропущенные данные о возрасте двумя способами

In [ ]:
# Способ 1: Средние арифметические
df['Age'].fillna(round(df['Age'].mean(), 2))

0      22.000000
1      38.000000
2      26.000000
3      35.000000
4      35.000000
5      29.699118
6      54.000000
7       2.000000
8      27.000000
9      14.000000
10      4.000000
11     58.000000
12     20.000000
13     39.000000
14     14.000000
15     55.000000
16      2.000000
17     29.699118
18     31.000000
19     29.699118
20     35.000000
21     34.000000
22     15.000000
23     28.000000
24      8.000000
25     38.000000
26     29.699118
27     19.000000
28     29.699118
29     29.699118
         ...    
861    21.000000
862    48.000000
863    29.699118
864    24.000000
865    42.000000
866    27.000000
867    31.000000
868    29.699118
869     4.000000
870    26.000000
871    47.000000
872    33.000000
873    47.000000
874    28.000000
875    15.000000
876    20.000000
877    19.000000
878    29.699118
879    56.000000
880    25.000000
881    33.000000
882    22.000000
883    28.000000
884    25.000000
885    39.000000
886    27.000000
887    19.000000
888    29.6991

In [ ]:
# Способ 2: Cредние арифметические в зависимости от класса каюты
age_1 = df[df['Pclass'] == 1]['Age'].mean()
age_2 = df[df['Pclass'] == 2]['Age'].mean()
age_3 = df[df['Pclass'] == 3]['Age'].mean()

In [ ]:
def fill_age(row):
    if pd.isnull(row['Age']):
        if row['Pclass'] == 1:
            return age_1
        if row['Pclass'] == 2:
            return age_2
        return age_3
    return row['Age']

In [ ]:
df['Age'] = df.apply(fill_age, axis = 1)

### Создаем новый столбец с информацией о том, был ли пассажир на борту один или с родственниками

In [ ]:
# Способ 1: с помощью именной функции и apply
def is_alone(row):
    if row['SibSp'] + row['Parch'] == 0:
        return 0
    return 1

In [ ]:
df['alone'] = df.apply(is_alone, axis = 1)

In [ ]:
# Способ 2: с помощью lambda-функции
df['alone'] = df.apply(lambda row: 1 if row.SibSp + row['Parch'] != 0 else 0, axis = 1)

In [ ]:
# Построим сводную таблицу (pivot_table), в которой отразим количество погибших и выживших 
# из числа путешествовавших в одиночку или с родственниками
df.pivot_table(index = 'Survived', columns = 'alone', values = 'Age', aggfunc = 'count')

alone,0,1
Survived,,
0,374,175
1,163,179


### Дополнительные приемы работы с числами

In [ ]:
new_df = pd.DataFrame({'col1': [1,2,3,4,5], 'col2': '2,5 4,5 3.3 1,5 2,9'.split()})

In [ ]:
new_df

,col1,col2
0,1,2.5
1,2,4.5
2,3,3.3
3,4,1.5
4,5,2.9


In [ ]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 2 columns):
col1    5 non-null int64
col2    5 non-null float64
dtypes: float64(1), int64(1)
memory usage: 160.0 bytes


In [ ]:
new_df['col2'].sum()

'2,54,53.31,52,9'

In [ ]:
new_df['col2'] = new_df['col2'].apply(lambda x: x.replace(',', '.'))

Три способа перевода значений в числовой формат:

In [ ]:
new_df['col2'] = pd.to_numeric(new_df['col2'])

In [ ]:
new_df['col2'] = new_df['col2'].astype('float64', inplace = True)

In [ ]:
new_df['col2'] = new_df['col2'].apply(lambda x: float(x))